# **Setup**

In [1]:
# download data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nckRRVYRiWG8VWoUY4wwtuQm6DjIbpmo' -O w_review_train.csv

# download font for matplot
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1DsfYE5uI1ZA_IXDNkezQTv-NQehxgZQy' -O THSarabun.ttf

# install library
!pip install gensim
!pip install pythainlp

--2024-07-25 12:01:07--  https://docs.google.com/uc?export=download&id=1nckRRVYRiWG8VWoUY4wwtuQm6DjIbpmo
Resolving docs.google.com (docs.google.com)... 2404:6800:4003:c11::65, 2404:6800:4003:c11::8b, 2404:6800:4003:c11::71, ...
Connecting to docs.google.com (docs.google.com)|2404:6800:4003:c11::65|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1nckRRVYRiWG8VWoUY4wwtuQm6DjIbpmo&export=download [following]
--2024-07-25 12:01:07--  https://drive.usercontent.google.com/download?id=1nckRRVYRiWG8VWoUY4wwtuQm6DjIbpmo&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 2404:6800:4003:c1c::84, 142.251.175.132
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|2404:6800:4003:c1c::84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60418516 (58M) [application/octet-stream]
Saving to: ‘w_review_train.csv’

w_review_train.csv  100%

In [2]:
from gensim.models import Word2Vec

from pythainlp import word_tokenize
from pythainlp.util import dict_trie
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import thai_stopwords

import pandas as pd
import numpy as np
import re

from sklearn.decomposition import PCA #reduce dimension for eg in SVD
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

%matplotlib inline

# **loading** **data**

In [ ]:
train = pd.read_csv("w_review_train.csv",  encoding='utf8', names=["sentence"])
train.head(10)

,sentence
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...


In [ ]:
train.shape

(40000, 1)

# **Creating model and training model using gensim**

In [ ]:
# add word to dict
words = ['บะหมี่แห้ง', 'ข้าวหมูกรอบ', 'ก๋วยเตี๋ยวหมูต้มยำ', 'ปลากระพงทอดน้ำปลา', 'ซูชิ', 'กาแฟ', 'ชานมไข่มุก', 'เค้กช็อกโกแลต', 'เครปเค๊ก', 'ไอศครีม']
custom_words_list = set(thai_words())
custom_words_list.update(words)
trie = dict_trie(dict_source=custom_words_list)

In [ ]:
STOP_WORD = list(thai_stopwords()) + [" ", "\n"]

FORMAT = r"[\u0E00-\u0E7Fa-zA-Z'0-9]+"

In [ ]:
def tokenize(sentence):
  return word_tokenize(sentence, engine="newmm", custom_dict=trie)

def cleaning_stop_word(tk_list):
  return [word for word in tk_list if word not in STOP_WORD]

def cleaning_symbols_emoji(tk_list):
  return [re.findall(FORMAT, text)[0] for text in tk_list if re.findall(FORMAT, text)]

**Word2Vec Function:**
* **tokenized_list** เป็นข้อมูลที่ใช้ train โมเดล โดยจะอยู่ในลักษณะของ list ของ list ของ tokens (คำในแต่ละรีวิว)

* **vector_size** เป็นจำนวนมิติของ embedding layer (ขนาดของ embedded  vector) ที่ต้องการ

* **window** คือจำนวนคำใกล้เคียงที่เกิดขึ้นร่วมกันกับคำที่กำลังฝึกฝนอยู่

* **min_count** คือ จำนวนขั้นต่ำของคำที่ปรากฎในข้อมูลการ train ทั้งหมด (คำที่ปรากฎน้อยจะไม่สนใจ)

* **worker** คือจำนวนของ computer threads ที่จะใช้ในการ train

* **sg** คือ ถ้า =0 คือ train ด้วย CBOW, =1 คือ train ด้วย Skip-gram (ปกติ Skip-gram จะใช้เวลา train นานกว่า CBOW)

* **epochs** คือ จำนวนรอบที่โมเดลจะ train ข้อมูล

In [ ]:
# model 1: doesn't clean anything

#### Your Code ####



In [ ]:
# model 2: only cleaning stopword

def tk_with_no_stop_word(sentence):
  return cleaning_stop_word( tokenize(sentence) )

#### Your Code ####


In [ ]:
# model 3: only cleaning symbols and emoji

def tk_with_no_symbols(sentence):
  return cleaning_symbols_emoji( tokenize(sentence) )

#### Your Code ####



In [ ]:
# model 4: cleaning stopword, symbols and emoji and more train wongnai to pretrained model

# load pretrained model
model_4 = Word2Vec.load('http://www.arts.chula.ac.th/ling/wp-content/uploads/TNCc5model.bin')

def big_cleaning(sentence):
  return  cleaning_symbols_emoji( cleaning_stop_word( tokenize(sentence) ) )

#### Your Code ####



In [ ]:
# word similarity
print(model_1.wv.similarity('อร่อย', 'สุดยอด'))
print(model_2.wv.similarity('อร่อย', 'สุดยอด'))
print(model_3.wv.similarity('อร่อย', 'สุดยอด'))
print(model_4.wv.similarity('อร่อย', 'สุดยอด'))

NameError: name 'model_1' is not defined

In [ ]:
print("doesn't clean anything:" , model_1.wv.most_similar("ชานมไข่มุก", topn=5))
print("only cleaning stopword:" , model_2.wv.most_similar("ชานมไข่มุก", topn=5))
print("only cleaing symbols and emoji:" , model_3.wv.most_similar("ชานมไข่มุก", topn=5))
print("cleaning stopword, symbols and emoji:" , model_4.wv.most_similar("ชานมไข่มุก", topn=5))

In [ ]:
print("doesn't clean anything:" , model_1.wv.most_similar("บะหมี่แห้ง", topn=5))
print("only cleaning stopword:" , model_2.wv.most_similar("บะหมี่แห้ง", topn=5))
print("only cleaing symbols and emoji:" , model_3.wv.most_similar("บะหมี่แห้ง", topn=5))
print("cleaning stopword, symbols and emoji:" , model_4.wv.most_similar("บะหมี่แห้ง", topn=5))

In [ ]:
print("doesn't clean anything:" , model_1.wv.most_similar('ปลากระพงทอดน้ำปลา', topn=5))
print("only cleaning stopword:" , model_2.wv.most_similar('ปลากระพงทอดน้ำปลา', topn=5))
print("only cleaing symbols and emoji:" , model_3.wv.most_similar('ปลากระพงทอดน้ำปลา', topn=5))
print("cleaning stopword, symbols and emoji:" , model_4.wv.most_similar('ปลากระพงทอดน้ำปลา', topn=5))

In [ ]:
print("doesn't clean anything:" , model_1.wv.most_similar('ซูชิ', topn=5))
print("only cleaning stopword:" , model_2.wv.most_similar('ซูชิ', topn=5))
print("only cleaing symbols and emoji:" , model_3.wv.most_similar('ซูชิ', topn=5))
print("cleaning stopword, symbols and emoji:" , model_4.wv.most_similar('ซูชิ', topn=5))

# **Visualization**


In [ ]:
plt.style.use("seaborn-notebook")

# setup font for matplotlib
thai_sarabun = FontProperties(fname="THSarabun.ttf", size=18)

In [ ]:
# using PCA

def display_plot(words):
  models = [model_1, model_2, model_3, model_4]
  fig, axes = plt.subplots(2, 2, figsize=(14, 14))

  j, k = 0, 0

  for modelx in models:
      if k == 2:
        k = 0
        j = 1

      word_vectors = np.array( [ modelx.wv[w] for w in words ])

      twodim = PCA().fit_transform(word_vectors)[:,:2]

      axes[j][k].scatter(twodim[:,0], twodim[:,1], c='salmon')
      for word, (x,y) in zip(words, twodim):
        axes[j][k].text(x+0.05, y+0.05, word, fontproperties=thai_sarabun)

      k += 1

  axes[0][0].set_title("Model 1: doesn't clean anything")
  axes[0][1].set_title("Model 2: only cleaning stopword")
  axes[1][0].set_title("Model 3: only cleaing symbols and emoji")
  axes[1][1].set_title("Model 4: cleaning stopword, symbols and emoji")


In [ ]:
#words = ['บะหมี่แห้ง', 'ข้าวหมูกรอบ', 'ก๋วยเตี๋ยวหมูต้มยำ', 'ปลากระพงทอดน้ำปลา', 'ชานมไข่มุก', 'เค้กช็อกโกแลต', 'เครปเค๊ก', 'ไอศครีม']

### Your Code for word list ###



In [ ]:
display_plot(words)